In [3]:
import logging
log_level = 25
logging.basicConfig(level=log_level) # root logger

import re
import pandas as pd

import sys
sys.path.append('E:/Code/chat/gdpr')


In [2]:
import fitz

import os
path = 'E:/Code/chat/gdpr/pdf/guidelines'
file_name = 'edpb_guidelines_2_2018_derogations_en.pdf'
paper_name = "Guidelines 2/2018 on derogations of Article 49 under Regulation 2016/679"
full_path = os.path.join(path, file_name)
doc = fitz.open(full_path)

In [5]:
import importlib
import src.extract_from_pdf
importlib.reload(src.extract_from_pdf)
from src.extract_from_pdf import output_doc_as_text

lines_to_delete = [] 
characters_to_replace = []
characters_to_replace.append(['“', '"'])
characters_to_replace.append(['”', '"'])
characters_to_replace.append(['‘', "'"])
characters_to_replace.append(['’', "'"])
characters_to_replace.append(['', "-"])
characters_to_replace.append(['–', "-"])




document_text = output_doc_as_text(doc, start_page = 2, end_page = 0, header_size=70, footer_size=70, lines_to_delete = lines_to_delete, characters_to_replace = characters_to_replace)

file = "../tmp/49_intl_transfer.md"
with open(file, 'w', encoding='utf-8') as f:
    f.write(document_text)

## Once you have the markdown file

In [5]:
import re
import pandas as pd
file_path = "../../original/article_49_intl_transfer.md"
with open(file_path, 'r', encoding = "utf-8") as file:
    text = file.read()

lines = text.split('\n')
# get rid of empty lines
lines = [line for line in lines if line]

doc_as_array = []
notes_as_array = []
#footnote_pattern = re.compile(r'^(\[\^\d{1,2}\]:)(.*)$')
footnote_pattern = re.compile(r'^\[\^(\d{1,2})\]:(.*)$')
for entry in lines:
    footnote_match = footnote_pattern.match(entry)
    if footnote_match:
        notes_as_array.append([footnote_match.group(1), footnote_match.group(2).strip()])
    else:
        doc_as_array.append(entry)

columns = ["note_number", "text"]
df_notes = pd.DataFrame(notes_as_array, columns = columns)

In [6]:
#columns = ["section", "subsection", "point", "subpoint", "heading", "text", "section_reference"]
columns = ["section", "subsection", "point", "heading", "text", "section_reference"]
section = ""
subsection = ""
point = ""
heading = False
text = ""
section_reference = ""
section_pattern = re.compile(r'(\d+)\.\s+(.*)')
subsection_pattern = re.compile(r'(\d\.\d+)\s+(.*)') 
point_pattern = re.compile(r'(\d\.\d\.\d+)\s+(.*)') 

data = []
table_data = []
for entry in doc_as_array:
    section_match = section_pattern.match(entry)
    subsection_match = subsection_pattern.match(entry)
    point_match = point_pattern.match(entry)
    if section_match:
        match = section_match
        section = match.group(1)
        subsection = ""
        point = ""
        heading = True
        text = match.group(2)
        section_reference = section
        data.append([section, subsection, point, heading, text, section_reference])
    elif subsection_match:
        match = subsection_match
        section = section
        subsection = match.group(1)
        point = ""
        heading = True
        text = match.group(2)
        section_reference = subsection
        data.append([section, subsection, point, heading, text, section_reference])
    elif point_match:
        match = point_match
        section = section
        subsection = subsection
        point = match.group(1)
        heading = True
        text = match.group(2)
        section_reference = point
        data.append([section, subsection, point, heading, text, section_reference])
    
    else:
        section = section
        subsection = subsection
        point = point
        heading = False
        text = entry
        section_reference = section_reference
        
        data.append(["", "", "", heading, text, section_reference])




df = pd.DataFrame(data, columns = columns)
# Remove my note about the table
#df = df[df["text"] != 'Note this table contains a column "References to BCR-C, application form BCR-C, and / or supporting documents[^14]" which is empty in the document because it is supposed to be filled out by the controller'] 

In [7]:
df[00:25]
#df[df["text"] == " "]

,section,subsection,point,heading,text,section_reference
0,1,,,True,GENERAL,1
1,,,,False,This document seeks to provide guidance as to ...,1
2,,,,False,The document builds on the previous work[^2] d...,1
3,,,,False,When applying Article 49 one must bear in mind...,1
4,,,,False,Article 49 (1) states that in the absence of a...,1
5,,,,False,"The WP29, as predecessor of the EDPB, has long...",1
6,,,,False,"Therefore, derogations under Article 49 are ex...",1
7,,,,False,When considering transferring personal data to...,1
8,,,,False,"Data exporters should also be aware that, in t...",1
9,,,,False,Occasional and not repetitive transfers,1


In [8]:
# Add footnotes
import re

def find_footnote_references(text):
    pattern = r'\[\^(\d+)\]'
    return re.findall(pattern, text)

for index, row in df.iterrows():
    footnotes = find_footnote_references(row['text'])
    if footnotes:
        augmented_note = row['text']
        for note in footnotes:
            augmented_note += f"\n[^{note}]: {df_notes[df_notes['note_number'] == note].iloc[0]['text']}"
        print(f"Row {index} augmented with footnotes")
        #print(augmented_note)
        df.at[index, "text"] = augmented_note
        #print

Row 1 augmented with footnotes
Row 2 augmented with footnotes
Row 4 augmented with footnotes
Row 5 augmented with footnotes
Row 6 augmented with footnotes
Row 16 augmented with footnotes
Row 21 augmented with footnotes
Row 22 augmented with footnotes
Row 25 augmented with footnotes
Row 31 augmented with footnotes
Row 32 augmented with footnotes
Row 34 augmented with footnotes
Row 38 augmented with footnotes
Row 41 augmented with footnotes
Row 42 augmented with footnotes
Row 44 augmented with footnotes
Row 46 augmented with footnotes
Row 54 augmented with footnotes
Row 56 augmented with footnotes
Row 57 augmented with footnotes
Row 59 augmented with footnotes
Row 61 augmented with footnotes
Row 63 augmented with footnotes
Row 65 augmented with footnotes
Row 66 augmented with footnotes
Row 74 augmented with footnotes
Row 78 augmented with footnotes
Row 90 augmented with footnotes
Row 97 augmented with footnotes
Row 102 augmented with footnotes
Row 103 augmented with footnotes
Row 104 aug

In [9]:
file = "../../inputs/documents/article_49_intl_transfer.parquet" # use parquet to deal with the complex text so I don't need to worry about escape characters
df.to_parquet(file, engine = 'pyarrow')

#df_no_table.to_csv(file, encoding = "utf-8", sep="|", index = False, na_rep="", quotechar='"')


## Check that the document class works as expected

In [13]:
import sys
sys.path.append('E:/Code/chat/gdpr')

import importlib
import gdpr_rag.documents.article_49_intl_transfer
importlib.reload(gdpr_rag.documents.article_49_intl_transfer)
from gdpr_rag.documents.article_49_intl_transfer import Article_49_Intl_Transfer

path_to_manual_as_csv_file = "../../inputs/documents/article_49_intl_transfer.parquet"

doc = Article_49_Intl_Transfer(path_to_manual_as_csv_file)


In [14]:
from IPython.display import Markdown, display

section = "1"

section = "2.1"
section = "2.1.3"

# section = "2.8"

rc = doc.reference_checker
print(rc.is_valid(section))


doc.get_heading(section)
doc.get_text(section)

print(doc.get_heading(section))
display(Markdown(doc.get_text(section)))


True
2 SPECIFIC INTERPRETATION OF THE PROVISIONS OF ARTICLE 49
2.1 The data subject has explicitly consented to the proposed transfer, after having been informed of the possible risks of such transfers for the data subject due to the absence of an adequacy decision and appropriate safeguards - Article (49 (1) (a))
2.1.3 Consent must be informed[^15]  particularly as to the possible risks of the transfer


# 2 SPECIFIC INTERPRETATION OF THE PROVISIONS OF ARTICLE 49

## 2.1 The data subject has explicitly consented to the proposed transfer, after having been informed of the possible risks of such transfers for the data subject due to the absence of an adequacy decision and appropriate safeguards - Article (49 (1) (a))

### 2.1.3 Consent must be informed[^15]  particularly as to the possible risks of the transfer

This condition is particularly important since it reinforces and further specifies the general requirement of "informed" consent as applicable to any consent and laid down in Art. 4 (11).[^16] As such, the general requirement of "informed" consent, requires, in the case of consent as a lawful basis pursuant to Article 6(1) (a) for a data transfer, that the data subject is properly informed in advance of the specific circumstances of the transfer, (i.e. the data controller's identity, the purpose of the transfer, the type of data, the existence of the right to withdraw consent, the identity or the categories of recipients).[^17]

In addition to this general requirement of "informed" consent, where personal data are transferred to a third country under Article 49 (1) (a), this provision requires data subjects to be also informed of the specific risks resulting from the fact that their data will be transferred to a country that does not provide adequate protection and that no adequate safeguards aimed at providing protection for the data are being implemented. The provision of this information is essential in order to enable the data subject to consent with full knowledge of these specific facts of the transfer and therefore if it is not supplied, the derogation will not apply.

The information provided to data subjects in order to obtain consent for the transfer of their personal data to third parties established in third countries should also specify all data recipients or categories of recipients, all countries to which the personal data are being transferred to, that the consent is the lawful  ground for the transfer, and that the third country to which the data will be transferred does not provide for an adequate level of data protection based on a European Commission decision.[^18] In addition, as mentioned above, information has to be given as to the possible risks for the data subject arising from the absence of adequate protection in the third country and the absence of appropriate safeguards. Such notice, which could be standardized, should include for example information that in the third country there might not be a supervisory authority and/or data processing principles and/or data subject rights might not be provided for in the third country.

In the specific case where a transfer is performed after the collection of personal data from the data subject has been made, the data exporter should inform the data subject of the transfer and of its risks before it takes place so as to collect his explicit consent to the "proposed" transfer.

As shown by the analysis above, the GDPR sets a high threshold for the use the derogation of consent.  This high threshold, combined with the fact that the consent provided by a data subject can be withdrawn at any time, means that consent might prove not to be a feasible long term solution for transfers to third countries.

  
[^15]: The general transparency requirements of Articles 13 and 14 of the GDPR should also be complied with. For more information see Guidelines on transparency under Regulation 2016/679 (WP 260)  
[^16]: See Article 29 Working Party Guidelines on Consent under Regulation 2016/679 (WP259)  
[^17]: Idem, page 13  
[^18]: The last mentioned requirement also stems from the duty to inform the data subjects (Article 13(1)(f), Article 14(1)(e))